## Descrição do Notebook

Neste notebook, os dados foram agregados por dia e as regras aplicadas estão descritas ao decorrer do código.

In [2]:
import pandas as pd 
import numpy as np
import xlrd
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv("../datasets/df_rpo_v3.csv")

In [4]:
df['DATA_TRAT.'] 

0         2017-01-12
1         2017-01-15
2         2017-01-16
3         2017-01-22
4         2017-02-12
             ...    
176587    2019-12-27
176588    2019-12-28
176589    2019-12-29
176590    2019-12-30
176591    2019-12-31
Name: DATA_TRAT., Length: 176592, dtype: object

In [5]:
# altera tipo do campo de object para datatime
df['DATA_TRAT.'] = pd.to_datetime(df["DATA_TRAT."], dayfirst = True)


In [6]:
df.shape

(176592, 13)

Retirando P10 e P90 das vendas dos produtos:

Criando o *Dataframe* agregado por semana

In [7]:
# Criando colunas de Semana e Ano para auxiliar na agregação
df.reset_index(inplace = True, drop = True)
#df["Semana"] = [x.isocalendar()[1] for x in df["DATA_TRAT."]]
df["Ano"] = df["DATA_TRAT."].dt.year
df["Mes"] = df['DATA_TRAT.'].dt.month
df["Dia"] = df["DATA_TRAT."].dt.day

In [8]:
df

,SAP+NOME,UF,DATA_TRAT.,Vendas Reais Qtde,Vendas Reais $,CMV,Margem Líquida Real %,Margem Líquida Real $,Venda Líquida Real $,Preço,preço2,CMV/QTD,Mg %,Ano,Mes,Dia
0,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-01-12,1.0,4.99000,1.75000,52.445652,1.93000,3.68,4.990000,4.99,1.750000,0.524457,2017,1,12
1,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-01-15,1.0,4.99000,1.81000,50.815217,1.87000,3.68,4.990000,4.99,1.810000,0.508152,2017,1,15
2,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-01-16,1.0,4.99000,1.81000,50.815217,1.87000,3.68,4.990000,4.99,1.810000,0.508152,2017,1,16
3,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-01-22,2.0,9.98000,3.62000,50.815217,3.74000,7.36,4.990000,4.99,1.810000,0.508152,2017,1,22
4,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2017-02-12,1.0,2.99000,1.81000,17.727273,0.39000,2.20,2.990000,2.99,1.810000,0.177273,2017,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176587,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,2019-12-27,39.0,193.11000,94.90000,32.575488,45.85000,140.75,4.951538,4.99,2.433333,0.325755,2019,12,27
176588,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,2019-12-28,36.0,188.91747,92.87859,34.527907,44.83794,129.86,5.247707,4.99,2.432778,0.345279,2019,12,28
176589,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,2019-12-29,32.0,168.87528,83.35593,34.528234,39.65913,114.86,5.277353,4.99,2.432187,0.345282,2019,12,29
176590,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,2019-12-30,26.0,136.22700,66.40200,34.777919,32.88600,94.56,5.239500,4.99,2.432308,0.347779,2019,12,30


### Removendo valores sem variação de preço ao longo do ano

> Fazendo as agregações: 
* Colunas de **Vendas Reais, Margem e CMV** foram somadas
* Colunas de **Preço  e Porcentagem de Margem** foram calculadas com base na média
* Coluna da razão **CMV/QTD** foi retirada uma nova fração entre a coluna CMV e QTD para respeitar a razão entre as grandezas.

In [9]:
df_agg = df[["SAP+NOME", "UF", "Ano", "Mes", "Dia",  "preço2", "Mg %",
                     "Margem Líquida Real %", "Preço"]].groupby(["SAP+NOME", "UF", "Dia","Mes", "Ano"]).mean().reset_index()

df_agg_sum = df[["SAP+NOME", "UF", "Ano", "Mes", "Dia", "Vendas Reais Qtde", "Vendas Reais $",
                     "Margem Líquida Real $", "Venda Líquida Real $", "CMV"]]. \
             groupby(["SAP+NOME", "UF", "Dia", "Mes", "Ano"]).sum().reset_index()
df_agg_sum["CMV/QTD"] = df_agg_sum["CMV"] / df_agg_sum["Vendas Reais Qtde"]

Unindo os dois *dataframes* de agregação criados para chegar ao *dataset* que será depositado no *Data Lake*

In [10]:
df_final = df_agg.copy()
df_final = df_final.merge(df_agg_sum, on = ["SAP+NOME", "UF", "Ano", "Mes", "Dia"], how = "inner")
df_final.reset_index(inplace = True, drop = True)

Retirando *P10* e *P90* dos produtos que possuiram variações de preço mais de 10 vezes. Essa agregação foi feita por **Produto** e por **UF** para eliminar os preços promocionais e as sazonalidades de vendas causadas por eles. E para as demais situações de um dado **Produto** ou **UF**, onde o preço não variou mais de 10 vezes, mas, ainda sim, há sazonalidade na quantidade de vendas semanais, também foram removidos os valores menores que *P10*, para remover quantidades insignificantes e *P90*, para remover valores que destoam.

In [11]:
lst_uf = df_final["UF"].unique()
lst_sap = df_final["SAP+NOME"].unique()

for uf in lst_uf:
    for sap in lst_sap:
        df_seg = df_final.loc[(df_final["UF"] == uf) & (df_final["SAP+NOME"] == sap)].copy()
        if df_seg.empty == True:
               continue
        else:
            # Se preço variar mais de 10 vezes, encontraremos o p10 e o p90 de valor e quantidade
            if len(df_seg["Preço"].unique()) > 10:
                q10_p = np.quantile(df_seg["Preço"].unique(), .10)
                q90_p = np.quantile(df_seg["Preço"].unique(), .90)
                q_10_qtd = np.quantile(df_seg["Vendas Reais Qtde"], .10)
                q_90_qtd = np.quantile(df_seg["Vendas Reais Qtde"], .90)
                
                # Atribuindo a 'ind', todos os valores que atendem as condições
                ind = df_seg.loc[((df_seg["Preço"] < q10_p)  | (df_seg["Preço"] > q90_p)) &
                                 ((df_seg["Vendas Reais Qtde"] < q_10_qtd) | (df_seg["Vendas Reais Qtde"] > q_90_qtd))]
                
                # Dropando index dos valores de 'ind'
                df_final = df_final.drop(ind.index)
            
            # Atribuindo a df_seg o valor de df_final ao final da condição
            df_seg = df_final.loc[(df_final["UF"] == uf) & (df_final["SAP+NOME"] == sap)].copy()
            
            # Pegando o p10 e o p90 da coluna Vendas Reais Qtde
            p10 = np.quantile(df_seg["Vendas Reais Qtde"].unique(), .10)
            p90 = np.quantile(df_seg["Vendas Reais Qtde"].unique(), .90)
            
            # Atribuindo a variável ind todos os index que são menores que o p10 e maiores que o p90
            ind = df_seg.loc[(df_seg["Vendas Reais Qtde"] < p10)  | (df_seg["Vendas Reais Qtde"] > p90)]
            
            # Removendo do DF todos os index que foram atribuidos a variável ind
            df_final = df_final.drop(ind.index)

In [12]:
df_final = df_final.sort_values(by = ["Ano", "Mes", "Dia", "UF"]).reset_index(drop = True)

In [13]:
df_final.to_csv("../datasets/df_rpo_v4_dia.csv", index = False)

In [14]:
df_final

,SAP+NOME,UF,Dia,Mes,Ano,preço2,Mg %,Margem Líquida Real %,Preço,Vendas Reais Qtde,Vendas Reais $,Margem Líquida Real $,Venda Líquida Real $,CMV,CMV/QTD
0,000000000004243420 - AMACIANTE COMFORT CONCENT...,AL,2,1,2017,7.99,0.407011,40.701149,7.99000,12.0,95.88000,35.41000,87.00,51.59000,4.299167
1,000000000004243420 - AMACIANTE COMFORT CONCENT...,AM,2,1,2017,7.99,0.554937,55.493749,7.86500,24.0,188.76000,104.75000,188.76,84.01000,3.500417
2,000000000004243420 - AMACIANTE COMFORT CONCENT...,AP,2,1,2017,7.99,0.499373,49.937312,7.99000,11.0,87.89000,39.83000,79.76,39.93000,3.630000
3,000000000004263231 - JG CAMA QUEEN PRATA 150FI...,AP,2,1,2017,129.99,0.498731,49.873123,129.99000,1.0,129.99000,47.17000,94.58,47.41000,47.410000
4,000000000002158151 - T VISITA P BORDA DESIRE A...,BA,2,1,2017,4.99,0.416721,41.672131,4.19000,10.0,41.90000,12.71000,30.50,17.79000,1.779000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104181,000000000002166665 - FRONHA AVULSA ESTAMPADA A...,TO,31,12,2019,4.99,0.425971,42.597137,5.34429,43.0,229.80447,66.49839,156.11,100.69542,2.341754
104182,000000000002166719 - FRONHA AVULSA LISA ANDREZA,TO,31,12,2019,4.99,0.412655,41.265475,4.99000,4.0,19.96000,6.00000,14.54,8.54000,2.135000
104183,000000000004261082 - KIT CABIDE C/3 MADEIRA REV,TO,31,12,2019,1.99,0.153832,15.383241,11.53950,14.0,161.55300,17.22000,111.94,100.31700,7.165500
104184,000000000004266282 - JG CAMA QUEEN MICROFIBRA ...,TO,31,12,2019,49.99,0.277426,27.742645,49.99000,2.0,99.98000,20.18000,72.74,52.56000,26.280000
